Miner Evaluation

To run a miner, read the miner.md located in assets/miner.md 
This notebook will demonstrate how to evaluate the results from testnet and compare it to expected results on mainnet.
Please ensure that your miner has been running for at least 2 hours before evaluating.


In [ ]:
!pip install wandb matplotlib

In [28]:
#INPUT DATA HERE
YOUR_TESTNET_UID = 10
TESTNET_RUN_ID = None
MAINNET_RUN_ID = None

In [29]:
# Locate your run on testnet
import wandb
import matplotlib.pyplot as plt
import pandas as pd

api = wandb.Api(timeout=120)
project_name = "macrocosmos/prompting-validators"
if TESTNET_RUN_ID is None:
    tag = "Netuid: 61"
    # Fetch all runs from the project
    runs = api.runs(project_name)

    # Filter runs that contain the specific tag
    filtered_runs = [run for run in runs if tag in run.tags]

    # Sort runs by created time, with the latest first
    if filtered_runs:
        latest_run_testnet = sorted(filtered_runs, key=lambda run: run.created_at, reverse=True)[0]
    else:
        print("No runs found with the specified tag. Try to manually locate the run using the Weights and Biases web app.")
        print("If there is no active run, please message us in discord and we will spin one up for you.")
else:
    latest_run_testnet = api.run(f"{project_name}/{TESTNET_RUN_ID}")




In [30]:
if MAINNET_RUN_ID is None:
    # Pull the most recent run from mainnet by taostats
    tag = "Netuid: 1"
    username = "taostats"

    # Fetch all runs from the project
    runs = api.runs(project_name)

    # Filter runs that contain the specific tag and are from the specified user
    filtered_runs = [run for run in runs if tag in run.tags and run.state == "running"]

    # Sort runs by created time, with the latest first
    if filtered_runs:
        latest_run_mainnet = sorted(filtered_runs, key=lambda run: run.created_at, reverse=True)[0]
    else:
        print("No runs found with the specified tag. Try to manually locate the most recent taostats run on mainnet using the Weights and Biases web app.")
else:
    latest_run_mainnet = api.run(f"{project_name}/{MAINNET_RUN_ID}")
¦

In [ ]:
testnet_df = pd.DataFrame(latest_run_testnet.scan_history()).explode(['uids', 'rewards'])
mainnet_df = pd.DataFrame(latest_run_mainnet.history()).explode(['uids', 'rewards'])

testnet_df = testnet_df[testnet_df['uids'] == YOUR_TESTNET_UID].tail(100)

grouped_testnet_df = testnet_df.groupby('task').rewards.mean()
grouped_mainnet_df = mainnet_df.groupby('task').rewards.mean()

# Plot the rewards grouped by task for both testnet and mainnet
plt.figure(figsize=(12, 6))
plt.bar(grouped_testnet_df.index, grouped_testnet_df, label='Testnet Rewards', alpha=0.3)
plt.bar(grouped_mainnet_df.index, grouped_mainnet_df, label='Mainnet Rewards', alpha=0.3)
plt.xlabel('Task')
plt.ylabel('Rewards')
plt.title('Rewards Comparison: Testnet vs Mainnet')
plt.legend()

In [ ]:
import vllm
from vllm import LLM, SamplingParams

In [ ]:
llm = LLM(model="casperhansen/qwen2-0.5b-instruct-awq", gpu_memory_utilization=0.2)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("casperhansen/qwen2-0.5b-instruct-awq")
model = AutoModelForCausalLM.from_pretrained("casperhansen/qwen2-0.5b-instruct-awq")

In [ ]:
from vllm import LLM, SamplingParams
llm = LLM(model="casperhansen/llama-3-8b-instruct-awq", gpu_memory_utilization=0.2)

In [2]:
from vllm.engine.arg_utils import AsyncEngineArgs
from vllm import LLM


llm = LLM(
    model="casperhansen/llama-3-8b-instruct-awq",
    tensor_parallel_size=1,
    pipeline_parallel_size=1,
    disable_custom_all_reduce=True,
    swap_space=4,
    max_num_batched_tokens=4096,
    trust_remote_code=False,
    max_model_len=2048,
    dtype="float16",
    enforce_eager=True,
    gpu_memory_utilization=0.5,
)
llm.generate("Who are you?")

/root/.cache/pypoetry/virtualenvs/prompting-fb5sw-i7-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2024-10-28 12:24:41,043	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


INFO 10-28 12:24:41 awq_marlin.py:77] The model is convertible to awq_marlin during runtime. Using awq_marlin kernel.
INFO 10-28 12:24:41 llm_engine.py:176] Initializing an LLM engine (v0.5.3.post1) with config: model='casperhansen/llama-3-8b-instruct-awq', speculative_config=None, tokenizer='casperhansen/llama-3-8b-instruct-awq', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=2048, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=True, quantization=awq_marlin, enforce_eager=True, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None), seed=0, served_model_name=casperhansen/llama-3-8b-instruct-awq, use_v2_block_manager=False, enab

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  50% Completed | 1/2 [00:00<00:00,  4.88it/s]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:01<00:00,  1.66it/s]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:01<00:00,  1.84it/s]



INFO 10-28 12:24:44 model_runner.py:692] Loading model weights took 5.3453 GB
INFO 10-28 12:24:45 gpu_executor.py:102] # GPU blocks: 7844, # CPU blocks: 2048


Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  4.23it/s, est. speed input: 17.08 toks/s, output: 68.30 toks/s]


[RequestOutput(request_id=0, prompt='Who are you?', prompt_token_ids=[15546, 527, 499, 30], prompt_logprobs=None, outputs=[CompletionOutput(index=0, text=' Aren’t you just some quote-unquote reporter wasting my time?”\nI held', token_ids=(74604, 1431, 499, 1120, 1063, 12929, 20486, 3022, 19496, 48897, 856, 892, 12671, 198, 40, 5762), cumulative_logprob=-43.68973688001279, logprobs=None, finish_reason=length, stop_reason=None)], finished=True, metrics=RequestMetrics(arrival_time=1730118289.7709765, last_token_time=1730118289.7709765, first_scheduled_time=1730118289.7766387, first_token_time=1730118289.8125596, time_in_queue=0.005662202835083008, finished_time=1730118290.0105848), lora_request=None)]

In [8]:
from transformers import pipeline

messages = [
    {"role": "user", "content": "Who are you?"},
]
pipe = pipeline("text-generation", model="casperhansen/llama-3-8b-instruct-awq", max_length=200, device="cuda")
tokenizer = AutoTokenizer.from_pretrained("casperhansen/llama-3-8b-instruct-awq")
messages_formatted = tokenizer.apply_chat_template(messages, tokenize=False)
pipe(messages_formatted)

`low_cpu_mem_usage` was None, now set to True since model is quantized.
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  4.82it/s]
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': "<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\nWho are you?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nI'm LLaMA, a large language model developed by Meta AI that can understand and respond to human input in a conversational manner. I'm trained on a massive dataset of text from the internet and can generate human-like responses to a wide range of topics and questions.\n\nI'm not a human, but rather a computer program designed to simulate conversation and answer questions to the best of my ability. My training data includes a vast amount of text from various sources, including books, articles, and websites, which allows me to learn about and generate responses to a wide range of topics.\n\nI'm constantly learning and improving my performance, so the more you chat with me, the better I'll become at understanding and responding to your questions and statements! What would you like to talk about?"}]

In [ ]:
messages_formatted

In [9]:
import torch
import numpy as np
import os
import random
from transformers import AutoModelForCausalLM, AutoTokenizer
from vllm import SamplingParams
import gc
import transformers
# Global variables for model and tokenizer
model = None
tokenizer = None

from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("casperhansen/qwen2-0.5b-instruct-awq")
model = AutoModelForCausalLM.from_pretrained("casperhansen/qwen2-0.5b-instruct-awq")

def set_seed(seed):
    """Set all seeds for reproducibility"""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)
    transformers.set_seed(seed)

def generate(prompt, seed):
    global model, tokenizer
    
    # Set all seeds
    set_seed(seed)
    
    # Tokenize input
    # inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    # # Generate
    # with torch.no_grad():
    #     outputs = model.generate(
    #         **inputs,
    #         max_new_tokens=200,
    #         do_sample=True,
    #         temperature=0.01,
    #         pad_token_id=tokenizer.pad_token_id,
    #         eos_token_id=tokenizer.eos_token_id,
    #     )
    
    # Decode and return
    # return tokenizer.decode(outputs[0], skip_special_tokens=True)
    return llm.generate(prompt, SamplingParams(seed=seed))[0].outputs[0].text
    # return pipe(prompt)

# Generate multiple times
for _ in range(10):
    result = generate(
        "1/ 🌟 What is Radix?Radix s a cutting-edge decentralized platform designed to defeat the biggest challenges in blockchain: scalability, security, and ease of use. But what sets it apart? Let's explore! #DeFi #Blockchain  #Radix",
        967594
    )
    print(result)

`low_cpu_mem_usage` was None, now set to True since model is quantized.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': "1/ 🌟 What is Radix?Radix s a cutting-edge decentralized platform designed to defeat the biggest challenges in blockchain: scalability, security, and ease of use. But what sets it apart? Let's explore! #DeFi #Blockchain  #RadixDLT 👑\nRadix solves some of the most pressing issues in blockchain, such as:\nScalability: Radix employs a novel architecture, using shards and a distributed ledger, to achieve incredible scalability, capable of processing 10,000+ transactions per second!\nSecurity: Radix boasts an adaptive, multi-layered security framework, ensuring unparalleled protection for your assets and data.\nEase of use: Radix boasts an intuitive interface, streamlining the process of joining, creating, and managing your own blockchain-based dApps.\nAnd that's not all! 🤯 Radix has also made significant strides in:\nCost reduction: Radix's energy-efficient design reduces costs by up to 90% compared to traditional"}]


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': "1/ 🌟 What is Radix?Radix s a cutting-edge decentralized platform designed to defeat the biggest challenges in blockchain: scalability, security, and ease of use. But what sets it apart? Let's explore! #DeFi #Blockchain  #RadixDLT 👑\nRadix solves some of the most pressing issues in blockchain, such as:\nScalability: Radix employs a novel architecture, using shards and a distributed ledger, to achieve incredible scalability, capable of processing 10,000+ transactions per second!\nSecurity: Radix boasts an adaptive, multi-layered security framework, ensuring unparalleled protection for your assets and data.\nEase of use: Radix boasts an intuitive interface, streamlining the process of joining, creating, and managing your own blockchain-based dApps.\nAnd that's not all! 🤯 Radix has also made significant strides in:\nCost reduction: Radix's energy-efficient design reduces costs by up to 90% compared to traditional"}]


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': "1/ 🌟 What is Radix?Radix s a cutting-edge decentralized platform designed to defeat the biggest challenges in blockchain: scalability, security, and ease of use. But what sets it apart? Let's explore! #DeFi #Blockchain  #RadixDLT 👑\nRadix solves some of the most pressing issues in blockchain, such as:\nScalability: Radix employs a novel architecture, using shards and a distributed ledger, to achieve incredible scalability, capable of processing 10,000+ transactions per second!\nSecurity: Radix boasts an adaptive, multi-layered security framework, ensuring unparalleled protection for your assets and data.\nEase of use: Radix boasts an intuitive interface, streamlining the process of joining, creating, and managing your own blockchain-based dApps.\nAnd that's not all! 🤯 Radix has also made significant strides in:\nCost reduction: Radix's energy-efficient design reduces costs by up to 90% compared to traditional"}]


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': "1/ 🌟 What is Radix?Radix s a cutting-edge decentralized platform designed to defeat the biggest challenges in blockchain: scalability, security, and ease of use. But what sets it apart? Let's explore! #DeFi #Blockchain  #RadixDLT 👑\nRadix solves some of the most pressing issues in blockchain, such as:\nScalability: Radix employs a novel architecture, using shards and a distributed ledger, to achieve incredible scalability, capable of processing 10,000+ transactions per second!\nSecurity: Radix boasts an adaptive, multi-layered security framework, ensuring unparalleled protection for your assets and data.\nEase of use: Radix boasts an intuitive interface, streamlining the process of joining, creating, and managing your own blockchain-based dApps.\nAnd that's not all! 🤯 Radix has also made significant strides in:\nCost reduction: Radix's energy-efficient design reduces costs by up to 90% compared to traditional"}]


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
